First, run 'docker-compose up'.

Check that elasticsearch is up and running:

In [12]:
from elasticsearch import Elasticsearch

client = Elasticsearch()
client.count()


{'count': 0,
 '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}}

Check that mongodb is up

In [13]:
import pymongo

client = pymongo.MongoClient("localhost", 27017)

client.server_info()

ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused